In [50]:
%run prepare_some_data.ipynb

Requirement already up-to-date: pymorphy2-dicts in c:\users\acer\anaconda3\lib\site-packages (2.4.393442.3710985)
Requirement already up-to-date: DAWG-Python in c:\users\acer\anaconda3\lib\site-packages (0.7.2)
Requirement already up-to-date: pymorphy2[fast] in c:\users\acer\anaconda3\lib\site-packages (0.8)
Requirement already up-to-date: nltk in c:\users\acer\anaconda3\lib\site-packages (3.5)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


великий


In [ ]:
Document = namedtuple('Document',['description','author','keywords',
                                  'title','body_a','body_h1',
                                  'body_h3','body_strong',
                                  'body_rest'])

In [22]:
def create_dict(words):
    counter=defaultdict(int)
    for word in words:
        counter[word]+=1
    return (dict(counter),len(words))
    

def create_docs():
    IDF_lem=defaultdict(int)
    docs=[None]*(len(get_id)+1)
    docs[0]=(dict(),Document(*[0]*9))

    i=0

    with open('bm_docs.txt','r', encoding = 'utf-8', errors = 'ignore') as f:
        for line in f:
            words=line.rstrip('\n').split('\t')
            doc_id=int(words[0])

            words=[words[0],*map(lambda string:' '.join(map(add_cash,
                                                            string.split())),
                                 words[1:])]

            words_description=words[1].split()

            words_author=words[2].split()

            words_keywords=words[3].split()

            words_title=words[4].split()

            words_a=words[5].split()
            words_h1=words[6].split()
            words_h3=words[7].split()
            words_strong=words[8].split()
            words_rest=words[9].split()


            all_words=set()
            feature=defaultdict(lambda:[0]*9)
            lens=[0]*9

            for ind,words in enumerate(
                [words_description,
                words_author,
                words_keywords,
                words_title,
                words_a,
                words_h1,
                words_h3,
                words_strong,
                words_rest
                 ]):
                for word in words:
                    feature[word][ind]+=1
                    all_words.add(word)
                lens[ind]+=len(words)


            for word in all_words:
                IDF_lem[word]+=1

            opt_feature=dict()
            for key in feature:
                opt_feature[key]=Document(*feature[key])

            docs[doc_id]=(opt_feature,Document(*lens))

            i+=1
            if i%500==0:
                print(i)
                
    return (docs,IDF_lem)


In [23]:
docs,IDF_lem=create_docs()

500


In [32]:
D_len=np.zeros((len(docs)-1,9))
for i,doc in enumerate(docs[1:]):
    D_len[i]=np.array([*doc[1]])

In [36]:
print(D_len[1])

[9.000e+00 0.000e+00 2.100e+01 9.000e+00 2.030e+02 9.400e+01 2.210e+02
 1.460e+02 3.399e+04]


In [46]:
def IDF(word,N,l):
    n=IDF_lem[word]
    return log(l+(N-n)/n)

def calculate_mean_length(docvec):
    mean_length=(D_len*np.array([docvec])).sum()
    return mean_length/len(get_id)

def BM25F(query,doc_id,docvec,mean_length,k1,b,l):
    score=0
    doc=docs[doc_id]
    dl=sum(map(lambda x,y:x*y,doc[1],docvec))
    for word in query:
        if word in doc[0]:
            f=sum(map(lambda x,y:x*y,doc[0][word],docvec))
            score+=(IDF(word,len(get_id),l)*
                    (k1+1)*f/(k1*(1-b+b*dl/mean_length)+f))
    return score


In [47]:
candidates,_=generate_candidates()

In [48]:
docvec=Document(description=10, author=25, 
                keywords=1, title=200, 
                body_a=1, body_h1=20, 
                body_h3=1, body_strong=1, 
                body_rest=1)

Document(description=5, author=11, keywords=3, title=7, body_a=1, body_h1=1, body_h3=1, body_strong=1, body_rest=1)

In [49]:
k1=500
b=0.75

l=2


mean_length=calculate_mean_length(docvec)
best_n=10

query_ext=dict()
with open('queries.fixed.txt',encoding='utf-8-sig') as f:
    with open('bm25res.txt','w') as out:         
        for i,line in enumerate(f):
            query_id,raw_query=line.split('\t')
            query=string_lem(raw_query.strip()).split()
        
            out.write(query_id+'\t')
            for cand in candidates[int(query_id)]:
                score,doc_id=(BM25F(query,cand,docvec,mean_length,k1,b,l),cand)
                out.write(str(score)+','+str(doc_id))
                out.write(' ')
            out.write('\n')

            if i%100==0:
                print(i)

0
100
200
300
